# Основное задание
Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data/kernels  
1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)  
2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)  
3.Добавьте аугментацию данных к пункту 2  
  
Сравните качество всех 3 полученных подходов  
  
# Задание со звездочкой  
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?  

import libraries

In [1]:
import torch
from torch import nn
import torchvision.models as models

import torchvision as tv
import time

import os

In [2]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
dev

device(type='cuda')

helpers

load data

In [4]:
data_dir = '../dll_hw_4_modern_cnn/hymenoptera_data/'

In [5]:
BATCH_SIZE = 32

In [6]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


create model

In [7]:
# import torchvision.models as models
# resnet18 = models.resnet18()
# alexnet = models.alexnet()
# vgg16 = models.vgg16()
# squeezenet = models.squeezenet1_0()
# densenet = models.densenet161()
# inception = models.inception_v3()
# googlenet = models.googlenet()
# shufflenet = models.shufflenet_v2_x1_0()
# mobilenet = models.mobilenet_v2()
# resnext50_32x4d = models.resnext50_32x4d()
# wide_resnet50_2 = models.wide_resnet50_2()
# mnasnet = models.mnasnet1_0()

In [8]:
# import torchvision.models as models
# resnet18 = models.resnet18(pretrained=True)
# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)

# 1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)

# ResNet 18

In [9]:
model = models.resnet18()

train model

In [10]:
model = model.to(dev)

In [11]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [12]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [13]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 3.645. Train acc: 0.000. Train Loss: 253.219
Step. time since epoch: 3.708. Train acc: 0.312. Train Loss: 255.453
Step. time since epoch: 3.771. Train acc: 0.500. Train Loss: 164.689
Step. time since epoch: 3.834. Train acc: 0.531. Train Loss: 142.716
Step. time since epoch: 3.899. Train acc: 0.438. Train Loss: 133.564
Step. time since epoch: 3.959. Train acc: 0.594. Train Loss: 86.072
Step. time since epoch: 4.015. Train acc: 0.656. Train Loss: 58.152
Step. time since epoch: 4.057. Train acc: 0.650. Train Loss: 35.578
epoch 1, loss 4.6289, train acc 0.451, test acc 0.124, time 6.7 sec
Step. time since epoch: 1.918. Train acc: 0.156. Train Loss: 871.940
Step. time since epoch: 2.081. Train acc: 0.562. Train Loss: 71.327
Step. time since epoch: 2.173. Train acc: 0.438. Train Loss: 49.724
Step. time since epoch: 2.234. Train acc: 0.406. Train Loss: 46.006
Step. time since epoch: 2.295. Train acc: 0.438. Train Loss: 25.063
Step. time since epoch: 2.358. Train acc: 

In [14]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16

In [15]:
model = models.vgg16()

train model

In [16]:
model = model.to(dev)

In [17]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [18]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [19]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.229. Train acc: 0.000. Train Loss: 221.575
Step. time since epoch: 2.440. Train acc: 0.562. Train Loss: 1652.128
Step. time since epoch: 2.652. Train acc: 0.531. Train Loss: 156.399
Step. time since epoch: 2.860. Train acc: 0.406. Train Loss: 3147.367
Step. time since epoch: 3.071. Train acc: 0.562. Train Loss: 190.053
Step. time since epoch: 3.281. Train acc: 0.500. Train Loss: 208.997
Step. time since epoch: 3.493. Train acc: 0.594. Train Loss: 201.230
Step. time since epoch: 3.652. Train acc: 0.500. Train Loss: 113.478
epoch 1, loss 24.1444, train acc 0.455, test acc 0.458, time 6.6 sec
Step. time since epoch: 2.272. Train acc: 0.562. Train Loss: 100.374
Step. time since epoch: 2.495. Train acc: 0.469. Train Loss: 400.974
Step. time since epoch: 2.704. Train acc: 0.438. Train Loss: 110.751
Step. time since epoch: 2.914. Train acc: 0.625. Train Loss: 140.192
Step. time since epoch: 3.122. Train acc: 0.500. Train Loss: 151.107
Step. time since epoch: 3.332. T

# 2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)